# Voting Prediction with Fuzzy System

**Political Analysis Example**

In [ ]:
!pip install pyfuzzy-toolbox -q

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from fuzzy_systems.core import LinguisticVariable
from fuzzy_systems.inference import MamdaniSystem

%matplotlib inline

### 1. Input Variables

In [ ]:
# Economic satisfaction: low, medium, high
economy = LinguisticVariable('economy', universe=(0, 10))
economy.add_term('low', 'triangular', (0, 0, 5))
economy.add_term('medium', 'triangular', (2, 5, 8))
economy.add_term('high', 'triangular', (5, 10, 10))

# Government approval: low, medium, high
approval = LinguisticVariable('approval', universe=(0, 10))
approval.add_term('low', 'triangular', (0, 0, 5))
approval.add_term('medium', 'triangular', (2, 5, 8))
approval.add_term('high', 'triangular', (5, 10, 10))

# Visualize
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 4))
economy.plot(ax=ax1, show=False)
ax1.set_title('Economic Satisfaction')
approval.plot(ax=ax2, show=False)
ax2.set_title('Government Approval')
plt.tight_layout()
plt.show()

### 2. Output Variable

In [ ]:
# Vote probability: low, medium, high
vote = LinguisticVariable('vote', universe=(0, 100))
vote.add_term('low', 'triangular', (0, 0, 50))
vote.add_term('medium', 'triangular', (20, 50, 80))
vote.add_term('high', 'triangular', (50, 100, 100))

vote.plot(figsize=(12, 4), show=True)

### 3. Fuzzy Rules

In [ ]:
# Create Mamdani system
fis = MamdaniSystem()
fis.add_input_variable(economy)
fis.add_input_variable(approval)
fis.add_output_variable(vote)

# Political rules
fis.add_rule("IF economy IS low AND approval IS low THEN vote IS low")
fis.add_rule("IF economy IS low AND approval IS medium THEN vote IS low")
fis.add_rule("IF economy IS medium AND approval IS low THEN vote IS low")
fis.add_rule("IF economy IS medium AND approval IS medium THEN vote IS medium")
fis.add_rule("IF economy IS high OR approval IS high THEN vote IS high")

print(f"Voting prediction system with {len(fis.rules)} rules")

### 4. Test Predictions

In [ ]:
# Test scenarios
scenarios = [
    {'economy': 2, 'approval': 3, 'desc': 'Low economy, low approval'},
    {'economy': 5, 'approval': 5, 'desc': 'Moderate both'},
    {'economy': 8, 'approval': 7, 'desc': 'Good economy, good approval'},
    {'economy': 3, 'approval': 8, 'desc': 'Low economy, high approval'},
    {'economy': 8, 'approval': 3, 'desc': 'High economy, low approval'}
]

for s in scenarios:
    inputs = {'economy': s['economy'], 'approval': s['approval']}
    result = fis.infer(inputs, method='centroid')
    print(f"{s['desc']:35s} → Vote probability: {result['vote']:5.1f}%")

### 5. Decision Surface

In [ ]:
# Generate surface
econ_range = np.linspace(0, 10, 30)
appr_range = np.linspace(0, 10, 30)
vote_surface = np.zeros((len(appr_range), len(econ_range)))

for i, a in enumerate(appr_range):
    for j, e in enumerate(econ_range):
        result = fis.infer({'economy': e, 'approval': a}, method='centroid')
        vote_surface[i, j] = result['vote']

# Contour plot
E, A = np.meshgrid(econ_range, appr_range)
plt.figure(figsize=(10, 8))
contour = plt.contourf(E, A, vote_surface, levels=15, cmap='RdYlGn')
plt.colorbar(contour, label='Vote Probability (%)')
plt.xlabel('Economic Satisfaction')
plt.ylabel('Government Approval')
plt.title('Voting Prediction Map')
plt.grid(True, alpha=0.3)
plt.show()

### 🎯 Exercise: Add More Variables

In [ ]:
# TODO: Add a third variable (e.g., 'security', 'healthcare')
# Create new LinguisticVariable
# Add it to the system
# Create new rules incorporating the third variable

# Example structure:
# security = LinguisticVariable('security', universe=(0, 10))
# security.add_term('low', 'triangular', (0, 0, 5))
# security.add_term('high', 'triangular', (5, 10, 10))
# fis.add_input_variable(security)
# fis.add_rule("IF economy IS high AND security IS high THEN vote IS high")